In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,990 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,545 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRel

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-27 00:25:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  3.09MB/s    in 0.3s    

2022-09-27 00:25:29 (3.09 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()

+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781|Minnetonka Men's ...| 2015-08-31|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 2015-08-31|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660|Anne Klein Perfec...| 2015-08-31|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263|adidas Men's 10K ...| 2015-08-31|          5|       

In [6]:
# Create the customers_table DataFrame
customers_df = review_df.groupby("customer_id").agg({"customer_id": 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16121565|             5|
|   48146680|             1|
|   11062912|             1|
|   51451778|             1|
|   27920838|             1|
|    4919528|             3|
|   47802851|             2|
|   42560427|             1|
|   29467780|             1|
|   49703087|            12|
|   38456994|             4|
|     557836|             1|
|   43783459|             1|
|   23607725|             1|
|   45632184|             1|
|   19979069|             2|
|   28380959|             1|
|   46563381|             1|
|   21628739|             3|
|   23042837|             2|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = review_df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0456787283|FX1 Small Adult A...|
|0456808574|Lantin White Viso...|
|0456830197|NVC Unisex Light ...|
|0456838384|Riz Small Unisex ...|
|0456840532|Max-MPH Clear - B...|
|0456846999|Blogger Eyewear S...|
|0456856293|Kismeth Eyewear C...|
|0456886095|Serelo Small Blac...|
|1428597565|Anne Michelle Lon...|
|345200015X|Mirage Sunglasses...|
|5949392884|KENDIYA Men's Gen...|
|6041696641|UKFS Designer Lad...|
|729300236X|Lantin Eyewear Un...|
|7293006209|LCS Mens Extra Da...|
|8037200124|Envirosax Kids Se...|
|8037200221|Envirosax Greengr...|
|8529304233|Boston Eyewear Wo...|
|8742245001|Small Convenient ...|
|8742245117|Hot Pink Canvas S...|
|8866133604|Moleskine Storage...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = review_df.select(["review_id","customer_id", "product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781| 2015-08-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853| 2015-08-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660| 2015-08-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263| 2015-08-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171| 2015-08-31|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457| 2015-08-31|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602| 2015-08-31|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055| 2015-08-31|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438| 2015-08-31|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213| 2015-08-31|
|R369CEXHXHC6NQ|   13409446|B00EYAFTCQ|     332158165| 2015-08-31|
|R171PPIJXFONVI|    9451727|B00I0QHY32|      49243908| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = review_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [25]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://telwoochallenge.czevtzc94kob.us-east-1.rds.amazonaws.com:5432/my_amazon_shoe_review"
config = {"user":"amazon", 
          "password": "amazonreview", 
          "driver":"org.postgresql.Driver"}

In [26]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [24]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [21]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [22]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)